In [33]:
import pandas as pd
import numpy as np
# Load the CSV file into a pandas DataFrame
file_path = 'lc_14to16.csv'  # Update with the correct path
df = pd.read_csv(file_path)

# Display the first few rows of the dataset to get an overview
print(df.head())


C:\Users\Lawry\AppData\Local\Temp\ipykernel_23024\3495072123.py:5: DtypeWarning: Columns (19,59,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


         id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  68407277        NaN     3600.0       3600.0           3600.0   36 months   
1  68355089        NaN    24700.0      24700.0          24700.0   36 months   
2  68341763        NaN    20000.0      20000.0          20000.0   60 months   
3  66310712        NaN    35000.0      35000.0          35000.0   60 months   
4  68476807        NaN    10400.0      10400.0          10400.0   60 months   

   int_rate  installment grade sub_grade  ... hardship_payoff_balance_amount  \
0     13.99       123.03     C        C4  ...                            NaN   
1     11.99       820.28     C        C1  ...                            NaN   
2     10.78       432.66     B        B4  ...                            NaN   
3     14.85       829.90     C        C5  ...                            NaN   
4     22.45       289.91     F        F1  ...                            NaN   

  hardship_last_payment_amount disbursement_

In [34]:
# Check the size of the dataset
print(f"Dataset contains {df.shape[0]} rows and {df.shape[1]} columns.")

# Display basic statistics of numerical columns
print(df.describe())

# Check for missing values
print(df.isnull().sum())

Dataset contains 1091131 rows and 151 columns.
                 id  member_id     loan_amnt   funded_amnt  funded_amnt_inv  \
count  1.091131e+06        0.0  1.091131e+06  1.091131e+06     1.091131e+06   
mean   5.814955e+07        NaN  1.495881e+04  1.495878e+04     1.495290e+04   
std    2.444354e+07        NaN  8.716084e+03  8.716085e+03     8.713271e+03   
min    5.571600e+04        NaN  1.000000e+03  1.000000e+03     7.750000e+02   
25%    4.021801e+07        NaN  8.000000e+03  8.000000e+03     8.000000e+03   
50%    6.147308e+07        NaN  1.305000e+04  1.305000e+04     1.302500e+04   
75%    7.717113e+07        NaN  2.000000e+04  2.000000e+04     2.000000e+04   
max    9.645316e+07        NaN  4.000000e+04  4.000000e+04     4.000000e+04   

           int_rate   installment    annual_inc           dti   delinq_2yrs  \
count  1.091131e+06  1.091131e+06  1.091131e+06  1.091066e+06  1.091131e+06   
mean   1.302811e+01  4.428704e+02  7.751796e+04  1.879634e+01  3.520109e-01   
std 

In [35]:
df['term'] = df['term'].str.strip().str.replace(' months', '').astype(int)

# Now, the 'term' column contains only integers like 36 and 60
print(df['term'].head())


0    36
1    36
2    60
3    60
4    60
Name: term, dtype: int64


In [36]:
# Drop irrelevant columns
df = df.drop(['id', 'member_id', 'url'], axis=1)


In [37]:
# Checking for missing values
missing_values = df.isnull().sum() / len(df) * 100
print(missing_values)

loan_amnt                 0.000000
funded_amnt               0.000000
funded_amnt_inv           0.000000
term                      0.000000
int_rate                  0.000000
                           ...    
settlement_status        97.507815
settlement_date          97.507815
settlement_amount        97.507815
settlement_percentage    97.507815
settlement_term          97.507815
Length: 148, dtype: float64


In [38]:
df = df.loc[:, missing_values < 50]
print(df.head())

   loan_amnt  funded_amnt  funded_amnt_inv  term  int_rate  installment grade  \
0     3600.0       3600.0           3600.0    36     13.99       123.03     C   
1    24700.0      24700.0          24700.0    36     11.99       820.28     C   
2    20000.0      20000.0          20000.0    60     10.78       432.66     B   
3    35000.0      35000.0          35000.0    60     14.85       829.90     C   
4    10400.0      10400.0          10400.0    60     22.45       289.91     F   

  sub_grade                    emp_title emp_length  ... percent_bc_gt_75  \
0        C4                      leadman  10+ years  ...              0.0   
1        C1                     Engineer  10+ years  ...              7.7   
2        B4                 truck driver  10+ years  ...             50.0   
3        C5  Information Systems Officer  10+ years  ...              0.0   
4        F1          Contract Specialist    3 years  ...             60.0   

   pub_rec_bankruptcies tax_liens tot_hi_cred_lim 

In [39]:
from sklearn.preprocessing import LabelEncoder

# Encode the target variable 'grade'
encoder = LabelEncoder()
df['grade_encoded'] = encoder.fit_transform(df['grade'])

# Drop the original 'grade' column
loan_data = df.drop('grade', axis=1)


In [40]:
from sklearn.preprocessing import LabelEncoder

# Convert all object-type columns to numeric using LabelEncoder
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le  # Store the encoders for future use (inverse transformation)


In [41]:
# Apply one-hot encoding for categorical variables
df = pd.get_dummies(df, drop_first=True)


In [42]:
print(df.dtypes)  # Ensure all columns are now numeric


loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                            int64
int_rate                      float64
                               ...   
total_il_high_credit_limit    float64
hardship_flag                   int64
disbursement_method             int64
debt_settlement_flag            int64
grade_encoded                   int64
Length: 93, dtype: object


In [43]:
# Check for missing values in the dataset
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])  # Display columns with missing values


dti                           65
inq_last_6mths                 1
mths_since_last_delinq    524671
revol_util                   551
avg_cur_bal                    6
bc_open_to_buy             11043
bc_util                    11673
mo_sin_old_il_acct         31128
mths_since_recent_bc       10418
mths_since_recent_inq     111774
num_rev_accts                  1
num_tl_120dpd_2m           48467
percent_bc_gt_75           11461
dtype: int64


In [44]:
# Step 1: Identify columns with more than 1000 missing values
missing_counts = df.isnull().sum()  # Count missing values per column

# Step 2: Filter out columns where missing values are greater than 1000
columns_to_drop = missing_counts[missing_counts > 500].index

# Step 3: Drop those columns
df = df.drop(columns=columns_to_drop)

missing_values = df.isnull().sum()
print(missing_values[missing_values > 0]) 


dti               65
inq_last_6mths     1
avg_cur_bal        6
num_rev_accts      1
dtype: int64


In [45]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
import pandas as pd

# Scale the data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Apply KNN Imputer
imputer = KNNImputer(n_neighbors=3)
df_imputed_scaled = imputer.fit_transform(df_scaled)

# Optionally inverse transform to original scale
df_imputed = scaler.inverse_transform(df_imputed_scaled)

# Convert the NumPy array back to a pandas DataFrame using the original column names
df_imputed = pd.DataFrame(df_imputed, columns=df.columns)

# Replace the original DataFrame with the imputed one
df = df_imputed


In [47]:
print(df['grade_encoded'].dtype)


float64


In [48]:
# Convert the 'grade_encoded' column to integer (if it's float or continuous)
df['grade_encoded'] = df['grade_encoded'].astype(int)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Ensure the 'grade_encoded' column is discrete
df['grade_encoded'] = df['grade_encoded'].astype(int)

# Step 1: Scale the data (assuming you already have missing values imputed)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.drop('grade_encoded', axis=1))

# Step 2: Apply Logistic Regression with an increased max_iter
model = LogisticRegression(max_iter=1000)  # Increase iterations to 1000

# Step 3: Apply RFE to select the top 10 features
rfe = RFE(model, n_features_to_select=10)
X_rfe = rfe.fit_transform(df_scaled, df['grade_encoded'])

# Step 4: Check the selected features
selected_features = df.drop('grade_encoded', axis=1).columns[rfe.support_]
print(selected_features)


In [55]:
print(selected_features)

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'issue_d', 'total_rec_int'],
      dtype='object')


In [58]:
df_full = pd.read_csv(file_path)

C:\Users\Lawry\AppData\Local\Temp\ipykernel_23024\953284202.py:1: DtypeWarning: Columns (19,59,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv(file_path)


In [60]:
numeric_columns = df_full.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = df_full.select_dtypes(include=['object']).columns

print(numeric_columns.shape)
print(categorical_columns.shape)

(116,)
(36,)


In [62]:
# Standardize numerical features
scaler = StandardScaler()
df_full[numeric_columns] = scaler.fit_transform(df_full[numeric_columns])

C:\Users\Lawry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\Lawry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\Lawry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [63]:
# Encode categorical features
le = LabelEncoder()
for col in categorical_columns:
    df_full[col] = le.fit_transform(df_full[col].astype(str))

In [67]:
# Standardize the specified columns
columns_for_outlier_detection = ['dti', 'annual_inc', 'delinq_2yrs']
df_outlier = df_full[columns_for_outlier_detection].copy()
df_outlier = (df_outlier - df_outlier.mean()) / df_outlier.std()

# Sum the standardized columns
df_outlier['sum'] = df_outlier.sum(axis=1)

# Calculate IQR
Q1 = df_outlier['sum'].quantile(0.25)
Q3 = df_outlier['sum'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = df_outlier[(df_outlier['sum'] < lower_bound) | (df_outlier['sum'] > upper_bound)]

# Calculate the number of rows to remove (1% of total rows)
rows_to_remove = int(0.01 * len(df_full))

# Sort outliers by their distance from the median
outliers['distance'] = abs(outliers['sum'] - outliers['sum'].median())
outliers_sorted = outliers.sort_values('distance', ascending=False)

# Select the top 1% of outliers
outliers_to_remove = outliers_sorted.head(rows_to_remove)

# Remove outliers from the original dataframe
df_cleaned = df_full[~df_full.index.isin(outliers_to_remove.index)]

print(f"Original dataset shape: {df_full.shape}")
print(f"Cleaned dataset shape: {df_cleaned.shape}")
print(f"Number of outliers removed: {len(df_full) - len(df_cleaned)}")

C:\Users\Lawry\AppData\Local\Temp\ipykernel_23024\938696781.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outliers['distance'] = abs(outliers['sum'] - outliers['sum'].median())


Original dataset shape: (771821, 152)
Cleaned dataset shape: (764103, 152)
Number of outliers removed: 7718
